<a href="https://colab.research.google.com/github/manojach87/miscellaneous/blob/master/Movie_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import difflib

In [7]:
csv="https://github.com/jgoodall/cinevis/raw/master/data/csvs/moviedata.csv"
csv="https://github.com/shubham1710/Movie-Recommendation-System/raw/master/moviedata.csv"

In [8]:
movie = pd.read_csv(csv)
movie

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,4798,220000,Action Crime Thriller,NaN,9367,united states\u2013mexico barrier legs arms pa...,es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,"[{""name"": ""Columbia Pictures"", ""id"": 5}]","[{""iso_3166_1"": ""MX"", ""name"": ""Mexico""}, {""iso...",1992-09-04,2040920,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238,Carlos Gallardo Jaime de Hoyos Peter Marquardt...,"[{'name': 'Robert Rodriguez', 'gender': 0, 'de...",Robert Rodriguez
4799,4799,9000,Comedy Romance,NaN,72766,NaN,en,Newlyweds,A newlywed couple's honey

In [9]:
features = ['keywords','cast','genres','director','tagline']
for feature in features:
    movie[feature] = movie[feature].fillna('')
    
def combine_features(row):
    try:
        return row['keywords'] +" "+row['cast']+" "+row['genres']+" "+row['director']+" "+row['tagline']
    except:
        print ("Error:", row)

movie["combined_features"] = movie.apply(combine_features,axis=1)

In [10]:
def title_from_index(index):
    return movie[movie.index == index]["title"].values[0]

def index_from_title(title):
    title_list = movie['title'].tolist()
    common = difflib.get_close_matches(title, title_list, 1)
    titlesim = common[0]
    return movie[movie.title == titlesim]["index"].values[0]

In [16]:
cv = CountVectorizer()
count_matrix = cv.fit_transform(movie["combined_features"])
cosine_sim = cosine_similarity(count_matrix) 

user_movie = input("Enter movie of your choice:\t")
movie_index = index_from_title(user_movie)

similar_movies =  list(enumerate(cosine_sim[movie_index]))
similar_movies_sorted = sorted(similar_movies,key=lambda x:x[1],reverse=True)
i=0
print("\nOther movies you might be interested in:-\n")
for rec_movie in similar_movies_sorted:
        if(i!=0):
            print (i,") ",title_from_index(rec_movie[0]),sep="")
        i=i+1
        if i>50:
            break

Enter movie of your choice:	ring

Other movies you might be interested in:-

1) Red State
2) Halloween 4: The Return of Michael Myers
3) Graduation Day
4) Bowling for Columbine
5) Marmaduke
6) For Your Consideration
7) Open Range
8) Severance
9) Drive Hard
10) Tin Can Man
11) Day of the Dead
12) Roger & Me
13) Halloween: The Curse of Michael Myers
14) The Hills Have Eyes 2
15) 30 Days of Night
16) The Collection
17) The Omega Code
18) Survival of the Dead
19) Below Zero
20) Police Academy: Mission to Moscow
21) A Haunted House
22) From Dusk Till Dawn
23) Tusk
24) Scream 4
25) A Lonely Place to Die
26) Nine Dead
27) The Chronicles of Narnia: The Voyage of the Dawn Treader
28) The Beyond
29) Tales from the Crypt: Demon Knight
30) Sicko
31) Paranormal Activity 3
32) Prison
33) Eye of the Dolphin
34) Independence Day: Resurgence
35) Megiddo: The Omega Code 2
36) A Mighty Wind
37) The Betrayed
38) Batman Begins
39) Exorcist: The Beginning
40) Parker
41) Funny Games
42) Pathology
43) Yoga Ho

In [199]:
query="jumanji"
def get_titles_from_imdb(query):
  URL="https://www.imdb.com/find?q="+query+"&ref_=nv_sr_sm"
  import requests
  from bs4 import BeautifulSoup

  page = requests.get(URL)
  soup = BeautifulSoup(page.content, 'html.parser')
  #page = requests.get(URL)
  results = soup.find_all(class_='findSection')
  titles=dict()
  for elem in results:
    h3_elem=elem.find("h3", class_="findSectionHeader")
    section_title=h3_elem.text.strip()
    if(section_title=="Titles"):
      print(section_title)
      for more_elem in elem.find_all(class_="result_text"):
        # Each job_elem is a new BeautifulSoup object.
        # You can use the same methods on it as you did before.
        #print(more_elem)
        a_elem = more_elem.find('a')
        imdb_id=a_elem["href"].replace("/title/","").replace("/","")
        print(imdb_id)
        link="https://www.imdb.com"+a_elem["href"]
        full_title=more_elem.text.strip()
        short_title=a_elem.text.strip()
        titles.update({full_title:[link,short_title,imdb_id]})
  return(titles)

In [200]:
pd.DataFrame(get_titles_from_imdb("john wick:"))

Titles
tt2911666
tt6146586
tt4425200


,"John Wick (2014) aka ""John Wick: Sin control""",John Wick: Chapter 3 - Parabellum (2019),John Wick: Chapter 2 (2017)
0,https://www.imdb.com/title/tt2911666/,https://www.imdb.com/title/tt6146586/,https://www.imdb.com/title/tt4425200/
1,John Wick,John Wick: Chapter 3 - Parabellum,John Wick: Chapter 2
2,tt2911666,tt6146586,tt4425200


In [201]:
#titles=
get_titles_from_imdb("john wick:")


Titles
tt2911666
tt6146586
tt4425200


{'John Wick (2014) aka "John Wick: Sin control"': ['https://www.imdb.com/title/tt2911666/',
  'John Wick',
  'tt2911666'],
 'John Wick: Chapter 2 (2017)': ['https://www.imdb.com/title/tt4425200/',
  'John Wick: Chapter 2',
  'tt4425200'],
 'John Wick: Chapter 3 - Parabellum (2019)': ['https://www.imdb.com/title/tt6146586/',
  'John Wick: Chapter 3 - Parabellum',
  'tt6146586']}

In [242]:
#def get_title_dtl_from_imdb(URL):
URL="https://www.imdb.com/title/tt2911666/"
URL="https://www.imdb.com/title/tt6146586/"
import requests
from bs4 import BeautifulSoup

page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')
#page = requests.get(URL)
results = soup.find_all(class_='findSection')

rating_elem = soup.find(class_='ratingValue')
rating = rating_elem.text.strip()
#print(rating)
reviews=rating_elem.find("strong")["title"]
#print(reviews)
title_elem=soup.find(class_='title_wrapper')
title_year_elem=title_elem.find(id="titleYear")
year=title_year_elem.find("a").text.strip()
title_year_elem.decompose()
title_name=title_elem.find("h1").text.strip()
#print(title_elem)
more_details_elem=title_elem.find(class_="subtext")
# get genres
genres=[]
for a_elem in more_details_elem.find_all("a"):
  if( "genres" in a_elem["href"]):
    genres.append(a_elem.text.strip())
    a_elem.decompose()

#print(genres)

movie_time=more_details_elem.find("time").text.strip()
more_details_elem.find("time").decompose()
release_info_elem=more_details_elem.find("a")
release_info=release_info_elem.text.strip()
release_info_elem.decompose()
#title_elem=title_elem.find()
#print(more_details_elem.findChildren())
for span in more_details_elem.find_all("span"):
  span.decompose()
maturity_rating=more_details_elem.text.strip().split()[0]
#print(more_details)

title_elem.decompose()

titles=dict({
      "title":title_name
    , "rating" : rating
    , "reviews":reviews
    , "year":year
    , "genres":genres
    , "movie_time":movie_time
    , "release_info":release_info
    , "maturity_rating":maturity_rating
})

credits_elem=soup.find_all(class_="credit_summary_item")

for creditor in credits_elem:
  creditor_type=creditor.find("h4").text.strip()
  creditor_names=[nm.text.strip() for nm in creditor.find_all("a")]
  titles.update({creditor_type:creditor_names})

summary_text=soup.find(class_="summary_text").text.strip()
titles.update({"summary_text":summary_text})

title_details_elem=soup.find(id="titleDetails")
for seemore_elems in title_details_elem.find_all(class_="see-more"):
  seemore_elems.decompose()

#print(credits_elem)
for title_detail in title_details_elem.find_all(class_="txt-block"):
  title_detail_type=title_detail.find("h4")
  if(title_detail_type is not None):
    title_detail_type1=title_detail_type.text.strip()
    title_detail_type.decompose()
    title_detail_type=title_detail_type1
  #print(title_detail_type)
  title_detail_text1=""
  if (title_detail.find_all("a") is not None):
    title_detail_text1=[nm.text.strip().replace("(estimated)","") for nm in title_detail.find_all("a")]
    [a.decompose() for a in title_detail.find_all("a")]
    [a.decompose() for a in title_detail.find_all(class_="ghost")]
  
  title_detail_text2=title_detail.text.strip()
  #titles.update({creditor_type:creditor_names})
  title_detail_text1.append(title_detail_text2)

  #print({str(title_detail_type).replace(":",""):str(title_detail_text1)})
  titles.update({str(title_detail_type).replace(":",""):str(title_detail_text1)})

#print(soup.find(id="titleDetails"))

#print(title_elem)
#return(titles)
titles

{'Also Known As': "['John Wick: Chapter 3 - Parabellum']",
 'Aspect Ratio': "['2.39 : 1']",
 'Budget': "['$75,000,000\\n            (estimated)']",
 'Color': "['Color', '']",
 'Country': "['USA', '']",
 'Cumulative Worldwide Gross': "['$326,709,727']",
 'Director:': ['Chad Stahelski'],
 'Filming Locations': "['New York City, New York, USA', '']",
 'Gross USA': "['$171,015,687']",
 'Language': "['English', 'Russian', 'Japanese', 'Indonesian', 'Mandarin', 'Italian', 'Arabic', 'Latin', '']",
 'None': "['IMDbPro', 'Show more on\\n  \\xa0»']",
 'Official Sites': "['Official Facebook', 'Official Site', '']",
 'Opening Weekend USA': "['$56,818,067,\\n19 May 2019']",
 'Production Co': "['Lionsgate', 'Summit Entertainment', '87Eleven', ',,']",
 'Release Date': "['17 May 2019 (USA)']",
 'Runtime': "['130 min']",
 'Sound Mix': "['Dolby Surround 7.1', 'Dolby Atmos', '']",
 'Stars:': ['Keanu Reeves',
  'Halle Berry',
  'Ian McShane',
  'See full cast & crew'],
 'Writers:': ['Derek Kolstad', 'Shay H